<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

*   Understand the dataset for Chicago Public School level performance
*   Store the dataset in an Db2 database on IBM Cloud instance
*   Retrieve metadata about tables and columns and query data from mixed case columns
*   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**:

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR.

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next".

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database


In [1]:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.7
!pip install sqlalchemy==1.3.9


  Using cached ibm_db-3.1.0-cp37-cp37m-linux_x86_64.whl
  Using cached ibm_db_sa-0.3.7-py3-none-any.whl
  Using cached SQLAlchemy-1.4.28-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached importlib_metadata-4.8.2-py3-none-any.whl (17 kB)
  Using cached greenlet-1.1.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (150 kB)
  Using cached typing_extensions-4.0.1-py3-none-any.whl (22 kB)
  Using cached zipp-3.6.0-py3-none-any.whl (5.3 kB)
  Attempting uninstall: zipp
    Found existing installation: zipp 3.6.0
    Uninstalling zipp-3.6.0:
      Successfully uninstalled zipp-3.6.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.0.1
    Uninstalling typing-extensions-4.0.1:
      Successfully uninstalled typing-extensions-4.0.1
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.

In [2]:
%load_ext sql

In [3]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://ljj84886:HIDDEN@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB?security=SSL

'Connected: ljj84886@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [4]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select * from syscat.tables where tabschema = 'LJJ84886'

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
LJJ84886,CARSALESTABLE,LJJ84886,U,T,N,None,None,None,None,2021-11-20 22:07:17.536004,2021-11-20 22:07:17.536004,2021-11-20 22:07:17.536004,None,5,4,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2021-11-20 22:07:17.536004,0,,None,None,N,LJJ84886,,,,2021-11-20,,N,R,N,-1.0,None
LJJ84886,PETSHOP,LJJ84886,U,T,N,None,None,None,None,2021-11-28 03:10:13.988547,2021-11-28 03:10:13.988547,2021-11-28 03:10:13.988547,None,5,5,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2021-11-28 03:10:13.988547,0,,None,None,N,LJJ84886,,,,2021-11-28,,N,R,N,-1.0,None
LJJ84886,BOOKSHOP,LJJ84886,U,T,N,None,None,None,None,2021-11-28 03:13:37.273817,2021-11-28 03:50:12.060973,2021-11-28 03:50:12.060980,None,5,6,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,1,0,0,1,1,0,1,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2021-11-28 03:13:37.273817,0,,None,None,N,LJJ84886,,,,2021-11-28,,N,R,N,-1.0,None
LJJ84886,BOOKSHOP_AUTHORDETAILS,LJJ84886,U,T,N,None,None,None,None,2021-11-28 03:47:30.059477,2021-11-28 03:49:49.731952,2021-11-28 03:50:12.056145,None,3,7,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,0,1,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2021-11-28 03:47:30.059477,0,,None,None,N,LJJ84886,,,,2021-11-28,,N,R,N,-1.0,None
LJJ84886,STAFF_LOCATIONS,LJJ84886,U,T,N,None,None,None,None,2021-12-02 01:05:30.036017,2021-12-02 01:05:30.036017,2021-12-02 01:05:30.036017,None,4,8,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2021-12-02 01:05:30.036017,0,,None,None,N,LJJ84886,,,,2021-12-02,,N,R,N,-1.0,None
LJJ84886,INSTRUCTOR,LJJ84886,U,T,N,None,None,None,None,2021-12-11 15:51:20.398801,2021-12-11 15:51:20.613876,2021-12-11 15:51:20.613890,None,5,16,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2021-12-11 15:51:20.398801,0,,None,None,N,LJJ84886,,,,2021-12-11,,N,R,N,-1.0,None
LJJ84886,INTERNATIONAL_STUDENT_TEST_SCORES,LJJ84886,U,T,N,None,None,None,None,2021-12-11 16:04:07.198376,2021-12-11 16:04:07.198376,2021-12-11 16:04:07.198376,None,4,17,204,-1,-1,-1,-1,-1,-1,-1,-1,LJJ84886SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIB

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [6]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from syscat.columns where tabname = 'SCHOOLS'

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


1
78


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [7]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select * from sysibm.syscolumns where tbname = 'SCHOOLS'

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


name,tbname,tbcreator,remarks,coltype,NULLS,codepage,dbcscodepg,length,scale,colno,colcard,high2key,low2key,avgcollen,keyseq,typename,typeschema,DEFAULT,longlength,logged,compact,nquantiles,nmostfreq,composite_codepage,partkeyseq,source_tabschema,source_tabname,hidden,GENERATED,inline_length,numnulls,avgcollenchar,sub_count,sub_delim_length,IDENTITY,compress,avgdistinctperpage,pagevarianceratio,implicitvalue,seclabelid,collationid,pctinlined,pctencoded,avgencodedcollen
SCHOOL_ID,SCHOOLS,LJJ84886,None,INTEGER,Y,0,0,4,0,0,-1,    ,    ,-1,None,INTEGER,SYSIBM,None,4,,,-1,-1,0,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,None,-1,-1,-1.0
NAME_OF_SCHOOL,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,64,0,1,-1,                                 ,                                 ,-1,None,VARCHAR,SYSIBM,None,64,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
"Elementary, Middle, or High School",SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,2,0,2,-1,  ,  ,-1,None,VARCHAR,SYSIBM,None,2,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
STREET_ADDRESS,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,29,0,3,-1,                             ,                             ,-1,None,VARCHAR,SYSIBM,None,29,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
CITY,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,7,0,4,-1,       ,       ,-1,None,VARCHAR,SYSIBM,None,7,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
STATE,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,2,0,5,-1,  ,  ,-1,None,VARCHAR,SYSIBM,None,2,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
ZIP_CODE,SCHOOLS,LJJ84886,None,INTEGER,Y,0,0,4,0,6,-1,    ,    ,-1,None,INTEGER,SYSIBM,None,4,,,-1,-1,0,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,None,-1,-1,-1.0
PHONE_NUMBER,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,14,0,7,-1,              ,              ,-1,None,VARCHAR,SYSIBM,None,14,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
LINK,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,78,0,8,-1,                                 ,                                 ,-1,None,VARCHAR,SYSIBM,None,78,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
NETWORK_MANAGER,SCHOOLS,LJJ84886,None,VARCHAR,Y,1208,1200,40,0,9,-1,                                 ,                                 ,-1,None,VARCHAR,SYSIBM,None,40,,,-1,-1,1208,0,None,None,,,0,-1,-1,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [8]:
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


1
462


Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [9]:
%sql select max(SAFETY_SCORE) from SCHOOLS

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


1
99


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [10]:
%sql select SCHOOL_ID,NAME_OF_SCHOOL from SCHOOLS where SAFETY_SCORE = (select max(SAFETY_SCORE) from SCHOOLS)

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


school_id,name_of_school
610038,Abraham Lincoln Elementary School
609799,Alexander Graham Bell Elementary School
610084,Annie Keller Elementary Gifted Magnet School
609820,Augustus H Burley Elementary School
610132,Edgar Allan Poe Elementary Classical School
609901,Edgebrook Elementary School
610073,Ellen Mitchell Elementary School
610066,James E McDade Elementary Classical School
609803,James G Blaine Elementary School
610033,LaSalle Elementary Language Academy


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [13]:
%sql select SCHOOL_ID,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE from SCHOOLS where AVERAGE_STUDENT_ATTENDANCE <> 'None' order by AVERAGE_STUDENT_ATTENDANCE desc limit 10

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


school_id,name_of_school,average_student_attendance
609959,John Charles Haines Elementary School,98.40%
610217,James Ward Elementary School,97.80%
610132,Edgar Allan Poe Elementary Classical School,97.60%
609842,Rachel Carson Elementary School,97.60%
610329,Orozco Fine Arts & Sciences Elementary School,97.60%
610084,Annie Keller Elementary Gifted Magnet School,97.50%
610060,Andrew Jackson Elementary Language Academy,97.40%
610298,Lenart Elementary Regional Gifted Center,97.40%
610515,Disney II Magnet School,97.30%
610207,John H Vanderpoel Elementary Magnet School,97.20%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [14]:
%sql select SCHOOL_ID,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE from SCHOOLS where AVERAGE_STUDENT_ATTENDANCE <> 'None' order by AVERAGE_STUDENT_ATTENDANCE asc limit 5

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


school_id,name_of_school,average_student_attendance
609702,Richard T Crane Technical Preparatory High School,57.90%
609871,Barbara Vick Early Childhood & Family Center,60.90%
609736,Dyett High School,62.50%
609727,Wendell Phillips Academy High School,63.00%
610389,Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [15]:
%sql select SCHOOL_ID,NAME_OF_SCHOOL,replace(AVERAGE_STUDENT_ATTENDANCE,'%','') as AVERAGE_STUDENT_ATTENDANCE from SCHOOLS where AVERAGE_STUDENT_ATTENDANCE <> 'None' order by AVERAGE_STUDENT_ATTENDANCE asc limit 5

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


school_id,name_of_school,average_student_attendance
609702,Richard T Crane Technical Preparatory High School,57.90
609871,Barbara Vick Early Childhood & Family Center,60.90
609736,Dyett High School,62.50
609727,Wendell Phillips Academy High School,63.00
610389,Orr Academy High School,66.30


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [17]:
%sql select SCHOOL_ID,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE from SCHOOLS where replace(AVERAGE_STUDENT_ATTENDANCE,'%','') < 70

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


school_id,name_of_school,average_student_attendance
609871,Barbara Vick Early Childhood & Family Center,60.90%
609674,Chicago Vocational Career Academy High School,68.80%
609736,Dyett High School,62.50%
609722,Manley Career Academy High School,66.80%
610389,Orr Academy High School,66.30%
609702,Richard T Crane Technical Preparatory High School,57.90%
609759,Roberto Clemente Community Academy High School,69.60%
609727,Wendell Phillips Academy High School,63.00%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [18]:
%sql select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,sum(COLLEGE_ENROLLMENT) as TOTAL_COLLEGE_ENROLLMENT from SCHOOLS group by COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


community_area_number,community_area_name,total_college_enrollment
1,ROGERS PARK,4068
2,WEST RIDGE,8197
3,UPTOWN,4388
4,LINCOLN SQUARE,4132
5,NORTH CENTER,7541
6,LAKE VIEW,7055
7,LINCOLN PARK,5615
8,NEAR NORTH SIDE,3362
9,EDISON PARK,910
10,NORWOOD PARK,6469


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [19]:
%sql select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,sum(COLLEGE_ENROLLMENT) as TOTAL_COLLEGE_ENROLLMENT from SCHOOLS group by COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME order by 3 asc limit 3

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


community_area_number,community_area_name,total_college_enrollment
36,OAKLAND,140
37,FULLER PARK,531
47,BURNSIDE,549


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [20]:
%sql select SCHOOL_ID,NAME_OF_SCHOOL,SAFETY_SCORE from SCHOOLS order by SAFETY_SCORE asc limit 3

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


school_id,name_of_school,safety_score
609819,Edmond Burke Elementary School,1
610108,Luke O'Toole Elementary School,5
610202,George W Tilton Elementary School,6


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [23]:
%%sql
select d.hardship_index
  from schools s
  join chicago_socioeconomic_data d on s.community_area_number = d.ca
 where s.college_enrollment = 4368

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


hardship_index
6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [24]:
%%sql
select d.hardship_index
  from schools s
  join chicago_socioeconomic_data d on s.community_area_number = d.ca
 where s.college_enrollment = (select max(college_enrollment) from schools)

 * ibm_db_sa://ljj84886:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


hardship_index
6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                        |
| ----------------- | ------- | ----------------- | ----------------------------------------- |
| 2021-07-09        | 2.4     | Malika            | Updated connection string                 |
| 2021-05-19        | 2.3     | Lakshmi Holla     | Updated question                          |
| 2021-04-20        | 2.2     | Malika            | Added the libraries                       |
| 2020-11-27        | 2.1     | Sannareddy Ramesh | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya           | Moved lab to course repo in GitLab        |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
